<a href="https://colab.research.google.com/github/cbalkig/Anomaly_Detection_in_Videos/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras_layer_normalization

In [2]:
import os
from os import listdir
from os.path import join, isdir
import numpy as np
from PIL import Image
import keras
from keras.layers import Conv2DTranspose, ConvLSTM2D, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras_layer_normalization import LayerNormalization
import matplotlib.pyplot as plt

working_directory = '/content/drive/MyDrive/AnomalyDetectionInVideos'


class Config:
    DATASET_PATH = os.path.join(working_directory, "files")
    BATCH_SIZE = 4
    EPOCHS = 5
    WIDTH = 256
    HEIGHT = 256
    FRAME_SIZE = 10
    MODEL_PATH = os.path.join(working_directory, "model.hdf5")


def get_clips_by_stride(stride, frames_list, sequence_size):
    """ For data augmenting purposes.
    Parameters
    ----------
    stride : int
        The distance between two consecutive frames
    frames_list : list
        A list of sorted frames of shape 256 X 256
    sequence_size: int
        The size of the lstm sequence
    Returns
    -------
    list
        A list of clips , FRAME_SIZE frames each
    """
    clips = []
    sz = len(frames_list)
    clip = np.zeros(shape=(sequence_size, 256, 256, 1))
    cnt = 0
    for start in range(0, stride):
        for i in range(start, sz, stride):
            clip[cnt, :, :, 0] = frames_list[i]
            cnt = cnt + 1
            if cnt == sequence_size:
                clips.append(clip)
                cnt = 0
    return clips

def get_training_set():
    """
    Returns
    -------
    list
        A list of training sequences of shape (NUMBER_OF_SEQUENCES,SINGLE_SEQUENCE_SIZE,FRAME_WIDTH,FRAME_HEIGHT,1)
    """
    clips = []
    threshold = 1000
    for f in sorted(listdir(Config.DATASET_PATH)):
        directory_path = join(Config.DATASET_PATH, f)
        if isdir(directory_path):
            count = len(listdir(directory_path))
            print(directory_path, count)
            if count < threshold:
              threshold = count
            for c in listdir(directory_path):
              if len(c) < 7:
                os.rename(join(directory_path, c), join(directory_path, c.zfill(7)))

    print("Min folder threshold will be", threshold)
    # loop over the training folders (Train000,Train001,..)
    for f in sorted(listdir(Config.DATASET_PATH)):
        directory_path = join(Config.DATASET_PATH, f)
        if isdir(directory_path):
            print("Processing", directory_path)
            all_frames = []
            count = 0
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(directory_path)):
                count = count + 1
                if count > threshold:
                    break
                img_path = join(directory_path, c)
                if str(img_path)[-3:] == "tif":
                    img = Image.open(img_path).resize((256, 256))
                    img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img)
            # get the FRAME_SIZE-frames sequences from the list of images after applying data augmentation
            for stride in range(1, 3):
                clips.extend(get_clips_by_stride(stride=stride, frames_list=all_frames, sequence_size=Config.FRAME_SIZE))
    return clips


def get_model(reload_model=True):
    """
    Parameters
    ----------
    reload_model : bool
        Load saved model or retrain it
    """
    if not reload_model:
        return load_model(Config.MODEL_PATH,custom_objects={'LayerNormalization': LayerNormalization})
    training_set = get_training_set()
    training_set = np.array(training_set)
    seq = Sequential()
    seq.add(TimeDistributed(Conv2D(128, (11, 11), strides=4, padding="same"), batch_input_shape=(None, Config.FRAME_SIZE, 256, 256, 1)))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(64, (5, 5), strides=2, padding="same")))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(32, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    seq.add(ConvLSTM2D(64, (3, 3), padding="same", return_sequences=True))
    seq.add(LayerNormalization())
    # # # # #
    seq.add(TimeDistributed(Conv2DTranspose(64, (5, 5), strides=2, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2DTranspose(128, (11, 11), strides=4, padding="same")))
    seq.add(LayerNormalization())
    seq.add(TimeDistributed(Conv2D(1, (11, 11), activation="sigmoid", padding="same")))
    print(seq.summary())
    seq.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=1e-4, decay=1e-5, epsilon=1e-6))
    seq.fit(training_set, training_set,
            batch_size=Config.BATCH_SIZE, epochs=Config.EPOCHS, shuffle=False)
    seq.save(Config.MODEL_PATH)
    return seq


if __name__ == '__main__':
    model = get_model()

/content/drive/MyDrive/AnomalyDetectionInVideos/files/001dbbb8-b749-4ce0-a90d-740b8a783f35 215
/content/drive/MyDrive/AnomalyDetectionInVideos/files/083e9a31-84d9-455d-91e7-d1b0921b4921 219
/content/drive/MyDrive/AnomalyDetectionInVideos/files/0b1aab89-cbce-47af-a426-693f45c19acc 195
/content/drive/MyDrive/AnomalyDetectionInVideos/files/1474d8ce-335a-4568-b786-aa01d763e941 218
/content/drive/MyDrive/AnomalyDetectionInVideos/files/1f51d251-79aa-4bb3-b9f3-01a8275dab63 211
/content/drive/MyDrive/AnomalyDetectionInVideos/files/2195f4e0-76c9-465f-9123-8261ae863112 214
/content/drive/MyDrive/AnomalyDetectionInVideos/files/25f477c3-6deb-4ae0-b0ca-1cf107e03dd9 196
/content/drive/MyDrive/AnomalyDetectionInVideos/files/2e900a0f-dc64-4b45-a4cf-5a4a1f92547b 238
/content/drive/MyDrive/AnomalyDetectionInVideos/files/320b7780-a1af-480f-a7cb-b8c5d29c1d4e 191
/content/drive/MyDrive/AnomalyDetectionInVideos/files/4bb580ce-85a2-493d-9956-4d2e86cd4141 231
/content/drive/MyDrive/AnomalyDetectionInVideos/fi

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
